In [1]:
import pandas as pd
import numpy as np
import requests
import sys
from utils.camara_obter_dados import obter_dados
from utils.dataframe_to_file import dataframe_to_file

In [2]:
# Pretty Print
def pp(df):
    for row in df.itertuples():
        print('{} {}/{} - {} ({})\n'.format(row.siglaTipo, row.numero, row.ano, row.ementa, row.id))

In [3]:
df_proposicoes = pd.read_csv(
    '../data/2018-07-31-camara-proposicoes-legislativo+executivo'
)

In [4]:
df_proposicoes_destaque = df_proposicoes.query(
    '(siglaTipo == \'PEC\' or siglaTipo == \'PFC\' or siglaTipo == \'PL\' or \
    siglaTipo == \'PLP\' or siglaTipo == \'PDC\' or siglaTipo == \'MPV\')\
    and ano > 2002'
)

df_proposicoes_destaque.index = df_proposicoes_destaque.reset_index().index
df_proposicoes_destaque

,ano,ementa,id,idTipo,numero,siglaTipo,uri
0,2003,"Altera dispositivos da Lei nº 7.501, de 27 de ...",103385,139,7511,PL,https://dadosabertos.camara.leg.br/api/v2/prop...
1,2003,"Prorroga a vigência da Lei nº 8.989, de 24 de ...",103442,139,7512,PL,https://dadosabertos.camara.leg.br/api/v2/prop...
2,2003,"Altera a Lei nº 7.070, de 20 de dezembro de 19...",103648,139,7514,PL,https://dadosabertos.camara.leg.br/api/v2/prop...
3,2003,Altera a redação do art. 55 da Constituição F...,104314,136,1,PEC,NaN
4,2003,"Altera o Decreto - Lei nº 2.848, de 7 de dezem...",104324,139,2,PL,NaN
5,2003,"Altera o art. 129, do Decreto - Lei nº 2.848, ...",104325,139,3,PL,NaN
6,2003,Proíbe a participação de agentes públicos poli...,104326,139,4,PL,NaN
7,2003,"Altera os arts. 1º e 20 da Lei nº 7.716, de 5 ...",104327,139,5,PL,NaN
8,2003,"Altera a Lei nº 8.112, de 11 de dezembro de 19...",104328,139,6,PL,NaN
9,2003,Dispõe sobre a criação de Programa de Orientaç...,104329,139,7,PL,NaN


In [5]:
from utils.camara_obter_dados import obter_detalhes
from utils.camara_obter_dados import map_proposicoes

In [10]:
df_proposicoes_detalhadas = pd.DataFrame()

In [11]:
falhas = []
total = len(df_proposicoes_destaque)
for proposicao in df_proposicoes_destaque[28724:].itertuples():
    sys.stdout.write('\rProposicao {} de {}'.format(proposicao.Index, total))
    sys.stdout.flush()

    while True:
        try:
            df_i = obter_detalhes(
                endpoint='/proposicoes',
                id=proposicao.id,
                map_response=map_proposicoes
            )

            if df_i.empty:
                raise Exception(proposicao.id)
            else:
                df_proposicoes_detalhadas = df_proposicoes_detalhadas.append(df_i)
                break
        except:
            falhas.append(proposicao.id)

Proposicao 38332 de 38333

In [12]:
len(falhas)

16003

In [13]:
df_proposicoes_detalhadas

,id,uri,siglaTipo,idTipo,numero,ano,ementa,dataApresentacao,uriOrgaoNumerador,uriUltimoRelator,...,descricaoTipo,ementaDetalhada,keywords,uriPropPrincipal,uriPropAnterior,uriPropPosterior,urlInteiroTeor,urnFinal,texto,justificativa
0,1513957,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2003,2015,Dispõe sobre a obrigatoriedade de as instituiç...,2015-06-18T15:12,None,None,...,Projeto de Lei,None,"Obrigatoriedade, instituição financeira, banco...",None,None,None,http://www.camara.gov.br/proposicoesWeb/prop_m...,None,None,None
0,1513959,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2004,2015,"Altera a Lei nº 8.078, de 11 de setembro de 19...",2015-06-18T15:15,None,None,...,Projeto de Lei,None,"Alteração, Código de Defesa do Consumidor, prá...",None,None,None,http://www.camara.gov.br/proposicoesWeb/prop_m...,None,None,None
0,1513969,https://dadosabertos.camara.leg.br/api/v2/prop...,PDC,408,123,2015,Susta os efeitos da Decisão da Diretoria Coleg...,2015-06-18T15:24,None,None,...,Projeto de Decreto Legislativo de Sustação de ...,None,"Sustação, Decisão, Diretoria Colegiada, Agênci...",None,None,None,http://www.camara.gov.br/proposicoesWeb/prop_m...,None,None,None
0,1513970,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2005,2015,"Altera o art. 115 do Decreto-Lei nº 2.848, de ...",2015-06-18T15:26,None,None,...,Projeto de Lei,None,"Alteração, Código Penal, crime, redução, prazo...",https://dadosabertos.camara.leg.br/api/v2/prop...,None,None,http://www.camara.gov.br/proposicoesWeb/prop_m...,None,None,None
0,1513974,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2006,2015,"Altera a Lei N.º 9.503, de 23 de setembro de 1...",2015-06-18T15:30,None,None,...,Projeto de Lei,None,"Alteração, Código de Trânsito Brasileiro, docu...",None,None,None,http://www.camara.gov.br/proposicoesWeb/prop_m...,None,None,None
0,1513975,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2007,2015,"Altera a Lei nº 7.116, de 29 de agosto de 1983...",2015-06-18T15:31,None,None,...,Projeto de Lei,None,"Alteração, Lei da Carteira de Identidade, emis...",None,None,None,http://www.camara.gov.br/proposicoesWeb/prop_m...,None,None,None
0,1513976,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2008,2015,Dá nova redação ao art. 59 da Consolidação das...,2015-06-18T15:32,None,None,...,Projeto de Lei,None,"Alteração, Consolidação das Leis do Trabalho (...",None,None,None,http://www.camara.gov.br/proposicoesWeb/prop_m...,None,None,None
0,1513977,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2009,2015,Acrescenta alínea ao art. 38 da Lei nº 4.117- ...,2015-06-18T15:32,None,None,...,Projeto de Lei,None,"Alteração, Código Brasileiro de Telecomunicaçõ...",None,None,None,http://www.camara.gov.br/proposicoesWeb/prop_m...,None,None,None
0,1513978,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2010,2015,"Acrescenta o art. 154-A a Lei nº 9.503, de 199...",2015-06-18T15:33,None,None,...,Projeto de Lei,None,"Alteração, Código de Trânsito Brasileiro, veíc...",https://dadosabertos.camara.leg.br/api/v2/prop...,None,None,http://www.camara.gov.br/proposicoesWeb/prop_m...,None,None,None
0,1513984,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,139,2011,2015,Cria a possibilidade de dispensa das atividade...,2015-06-18T15:42,None,None,...,Projeto de Lei,None,"Alteração, Consolidação das Leis do Trabalho (...",https://dadosabertos.camara.leg.br/api/v2/prop...,None,None,http://www.camara.gov.br/proposicoesWeb/prop_m...,None,None,None


In [14]:
dataframe_to_file(
    dataframe=df_proposicoes_destaque,
    nome='proposicoes-detalhadas-apartirde28724',
    origem_dados='camara'
)